In [1]:
import pymaid
import navis

import numpy as np
import pandas as pd

import bokeh
import hvplot.pandas
import holoviews as hv

import bokeh.palettes
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, FactorRange, Legend
from bokeh.transform import dodge, factor_cmap
from bokeh.layouts import column
from bokeh.io import export_png

output_notebook()

Loading BokehJS ...

In [2]:
# !pip install selenium # required to export bokeh plots as files

In [3]:
# server="https://neurophyla.mrc-lmb.cam.ac.uk/catmaid/drosophila/l1/seymour/"
# api_token=""
# http_user=""
# http_password=""

In [4]:
# rm = pymaid.CatmaidInstance(server=server,
#                             api_token=api_token,
#                             http_user=http_user, # omit if not required
#                             http_password=http_password) # omit if not required


# Wild Type - Seymour

In [5]:
# Load neurons from Wild Type volumes
navis.set_pbars(jupyter=False)
pymaid.set_pbars(jupyter=False)

# Connect to the VFB CATMAID server hosting the FAFB data
wt = pymaid.connect_catmaid(server="https://l1em.catmaid.virtualflybrain.org/", api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {wt.catmaid_version}')

# Get connectors of neurons of interest
neurons_wt = pymaid.find_neurons(annotations='Valdes-Aleman et al. 2021')
connectors_wt = pymaid.get_connectors_between(neurons_wt, neurons_wt)
connectors_wt 

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


Server is running CATMAID version 2020.02.15-102-gce1dee4


INFO  : Found 37 neurons matching the search parameters (pymaid)                                                                                                                                                      


connector_id                 connector_loc  node1_id  source_neuron  \
0           478360  [63699.4, 83474.6, 119700.0]    478355          11995   
1           988063  [61681.6, 85329.0, 118950.0]    988060          11995   
2           988063  [61681.6, 85329.0, 118950.0]    988060          11995   
3          3543174  [46059.8, 83429.0, 114650.0]   3543194          11995   
4          4420609  [48474.7, 81430.2, 129950.0]   4420615          11995   
...            ...                           ...       ...            ...   
1416       7168246  [48130.8, 83043.3, 128300.0]   7168252       10804041   
1417      10812818  [66956.0, 87392.4, 129850.0]  10812793       10804041   
1418      12564988  [45216.2, 81931.8, 127250.0]  10816842       10804041   
1419      12997665  [61677.8, 84724.8, 129250.0]  10804942       10804041   
1420      20376232  [61727.0, 85158.1, 128150.0]  10805039       10804041   

      confidence1 creator1                     node1_loc  node2_id  \
0               5      NaN  [63809.6, 83676.0, 119700.0]    478346   
1               5      NaN  [61810.8, 85416.4, 118950.0]    988085   
2               5      NaN  [61810.8, 85416.4, 118950.0]    988069   
3               5      NaN  [46002.8, 83516.4, 114650.0]   3543173   
4               5      NaN  [48343.6, 81299.1, 129950.0]   4420607   
...           ...      ...                           ...       ...   
1416            5      NaN  [48184.0, 83345.4, 128300.0]   6290267   
1417            5      NaN  [66774.5, 87495.0, 129850.0]   7202182   
1418            5      NaN  [45252.9, 82021.7, 127250.0]   4413776   
1419            5      NaN  [61794.6, 84702.0, 129250.0]  12997662   
1420            5      NaN  [61804.1, 85218.8, 128150.0]   7595545   

      target_neuron  confidence2 creator2                     node2_loc  
0            471705            5      NaN  [63570.2, 83596.2, 119800.0]  
1           3034133            5      NaN  [61522.0, 85051.6, 118950.0]  
2           6207232            5      NaN  [61913.4, 85154.2, 118950.0]  
3           3282869            5      NaN  [46266.9, 83221.9, 114650.0]  
4           7599503            5      NaN  [48260.9, 81805.5, 130000.0]  
...             ...          ...      ...                           ...  
1416        3282869            5      NaN  [47871.4, 82904.1, 128250.0]  
1417        7195651            5      NaN  [66824.9, 87137.8, 129800.0]  
1418        7599503            5      NaN  [45079.4, 81994.5, 127300.0]  
1419        7599503            5      NaN  [61666.4, 85028.8, 129250.0]  
1420         471720            5      NaN  [61743.4, 84493.3, 128150.0]  

[1421 rows x 12 columns]

In [6]:
# Unique confidence scores for node1 and node2
print(connectors_wt["confidence1"].unique())
print(connectors_wt["confidence2"].unique())


[5]
[5 4]


In [7]:
# Function to compute Euclidean distance
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((np.array(point1) - np.array(point2)) ** 2))

In [8]:
#Find connectors below 100nm

# Compute connector-node2 distances and find those smaller than 100 nm
threshold_nm = 100
distances = connectors_wt.apply(lambda row: euclidean_distance(row['connector_loc'], row['node2_loc']), axis=1)
connectors_wt.loc[:, 'c_n2_distance'] = distances
small_distances = distances < threshold_nm

# Filter DataFrame for distances smaller than 100 nm
wt_filtered_df = connectors_wt[small_distances]
print(f"num. of connector-node2 distances smaller than {threshold_nm}nm: {len(wt_filtered_df)}")
# print(f"filtered df with distances smaller than {threshold_nm}nm: {wt_filtered_df.head()}")

# Calculate max, min, mean, and median
max_distance = connectors_wt['c_n2_distance'].max()
min_distance = connectors_wt['c_n2_distance'].min()
mean_distance = connectors_wt['c_n2_distance'].mean()
median_distance = connectors_wt['c_n2_distance'].median()

print(f"Max distance connector-node2: {max_distance} nm")
print(f"Min distance connector-node2: {min_distance} nm")
print(f"Mean distance connector-node2: {mean_distance} nm")
print(f"Median distance connector-node2: {median_distance} nm")

num. of connector-node2 distances smaller than 100nm: 9
Max distance connector-node2: 1553.487872498528 nm
Min distance connector-node2: 65.74990494289328 nm
Mean distance connector-node2: 291.72671731101406 nm
Median distance connector-node2: 258.91319008501614 nm


In [9]:
# Compute node1-node2 distances and find those smaller than 100 nm
threshold_nm = 100
distances = connectors_wt.apply(lambda row: euclidean_distance(row['node1_loc'], row['node2_loc']), axis=1)
connectors_wt.loc[:, 'n1_n2_distance'] = distances
small_distances = distances < threshold_nm

# Filter DataFrame for distances smaller than 100 nm
wt_filtered_df = connectors_wt[small_distances]
print(f"num. of connector-node2 distances smaller than {threshold_nm}nm: {len(wt_filtered_df)}")
# print(f"filtered df with distances smaller than {threshold_nm}nm: {wt_filtered_df.head()}")

# Calculate max, min, mean, and median
max_distance = connectors_wt['n1_n2_distance'].max()
min_distance = connectors_wt['n1_n2_distance'].min()
mean_distance = connectors_wt['n1_n2_distance'].mean()
median_distance = connectors_wt['n1_n2_distance'].median()

print(f"Max distance node1-node2: {max_distance} nm")
print(f"Min distance node1-node2: {min_distance} nm")
print(f"Mean distance node1-node2: {mean_distance} nm")
print(f"Median distance node1-node2: {median_distance} nm")


num. of connector-node2 distances smaller than 100nm: 2
Max distance node1-node2: 1698.6964325623323 nm
Min distance node1-node2: 89.30000000000291 nm
Mean distance node1-node2: 442.0992777118431 nm
Median distance node1-node2: 406.60748886364553 nm


# TNT VOLUME

In [10]:
#TNT Volume# Connect to the VFB CATMAID server hosting the FAFB data
pymaid.clear_cache()
tnt = pymaid.connect_catmaid(server="https://iav-tnt.catmaid.virtualflybrain.org/", project_id=2, api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {tnt.catmaid_version}')
neurons_tnt = pymaid.find_neurons(annotations='Valdes-Aleman et al. 2021')

# Get connectors of neurons of interest
connectors_tnt = pymaid.get_connectors_between(neurons_tnt, neurons_tnt)
connectors_tnt 

INFO  : Cached cleared. (pymaid)
INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


Server is running CATMAID version 2020.02.15-102-gce1dee4


INFO  : Found 36 neurons matching the search parameters (pymaid)                                                                                                                                                      


connector_id                connector_loc  node1_id  source_neuron  \
0        19272919  [57813.4, 26462.7, 15880.0]  19272920       19243093   
1        19272932  [57285.6, 26135.5, 15400.0]  19272933       19243093   
2        19272932  [57285.6, 26135.5, 15400.0]  19272933       19243093   
3        19272955  [57855.8, 26723.2, 15480.0]  19272956       19243093   
4        19272955  [57855.8, 26723.2, 15480.0]  19272956       19243093   
..            ...                          ...       ...            ...   
746      19287943   [53010.5, 24947.6, 6240.0]  19442479       19405635   
747      19442515   [53847.9, 24936.4, 6720.0]  19442516       19405635   
748      19442558   [53741.4, 24991.1, 8680.0]  19442559       19405635   
749      19287984  [53320.0, 24948.6, 10080.0]  19442581       19405635   
750      19287984  [53320.0, 24948.6, 10080.0]  19442581       19405635   

     confidence1 creator1                    node1_loc  node2_id  \
0              5      NaN  [57828.7, 26647.5, 15880.0]  19272925   
1              5      NaN  [57566.5, 26417.2, 15400.0]  19399331   
2              5      NaN  [57566.5, 26417.2, 15400.0]  19393573   
3              5      NaN  [57631.4, 26483.3, 15480.0]  19272957   
4              5      NaN  [57631.4, 26483.3, 15480.0]  19272958   
..           ...      ...                          ...       ...   
746            5      NaN   [53021.4, 24835.6, 6240.0]  19287942   
747            5      NaN   [53796.1, 24714.6, 6720.0]  19442521   
748            5      NaN   [53825.5, 25027.9, 8680.0]  19417516   
749            5      NaN  [53162.5, 25120.7, 10080.0]  19287985   
750            5      NaN  [53162.5, 25120.7, 10080.0]  19285839   

     target_neuron  confidence2 creator2                    node2_loc  
0         19369066            5      NaN  [58130.2, 26419.5, 16160.0]  
1         19369066            5      NaN  [56921.3, 26085.1, 15200.0]  
2         19363858            5      NaN  [56700.8, 25722.9, 15400.0]  
3         19369066            5      NaN  [58199.2, 26909.2, 15480.0]  
4         19363858            5      NaN  [58188.6, 26643.6, 15480.0]  
..             ...          ...      ...                          ...  
746       19259770            5      NaN   [53111.3, 25091.6, 6240.0]  
747       19259770            5      NaN   [53687.3, 24911.2, 6840.0]  
748       19386927            5      NaN   [53560.8, 24954.0, 8520.0]  
749       19259770            5      NaN  [53588.7, 25058.0, 10080.0]  
750       19258184            5      NaN   [53512.1, 25094.7, 9880.0]  

[751 rows x 12 columns]

In [11]:
# unique confidences for node1 and node2
print(connectors_tnt["confidence1"].unique())
print(connectors_tnt["confidence2"].unique())

[5]
[5]


In [12]:
#Find connectors below 100nm

# Compute distances and find those smaller than 100 nm
threshold_nm = 100
distances = connectors_tnt.apply(lambda row: euclidean_distance(row['connector_loc'], row['node2_loc']), axis=1)
connectors_tnt.loc[:, 'c_n2_distance'] = distances
small_distances = distances < threshold_nm

# Filter DataFrame for distances smaller than 100 nm
tnt_filtered_df = connectors_tnt[small_distances]
print(f"num. of connector-node2 distances smaller than {threshold_nm}nm: {len(tnt_filtered_df)}")

# Calculate max, min, mean, and median
max_distance = connectors_tnt['c_n2_distance'].max()
min_distance = connectors_tnt['c_n2_distance'].min()
mean_distance = connectors_tnt['c_n2_distance'].mean()
median_distance = connectors_tnt['c_n2_distance'].median()

print(f"Max distance connector-node2: {max_distance} nm")
print(f"Min distance connector-node2: {min_distance} nm")
print(f"Mean distance connector-node2: {mean_distance} nm")
print(f"Median distance connector-node2: {median_distance} nm")

num. of connector-node2 distances smaller than 100nm: 4
Max distance connector-node2: 1613.7975151796486 nm
Min distance connector-node2: 86.72266139827751 nm
Mean distance connector-node2: 357.2093880866985 nm
Median distance connector-node2: 320.65671675484936 nm


In [13]:
# Find distances between node1 and node2

# Compute distances and find those smaller than 100 nm
threshold_nm = 100
distances = connectors_tnt.apply(lambda row: euclidean_distance(row['node1_loc'], row['node2_loc']), axis=1)
connectors_tnt.loc[:, 'n1_n2_distance'] = distances
small_distances = distances < threshold_nm

# Filter DataFrame for distances smaller than 100 nm
tnt_filtered_df = connectors_tnt[small_distances]
len(tnt_filtered_df)


# Calculate max, min, mean, and median
max_distance = connectors_tnt['n1_n2_distance'].max()
min_distance = connectors_tnt['n1_n2_distance'].min()
mean_distance = connectors_tnt['n1_n2_distance'].mean()
median_distance = connectors_tnt['n1_n2_distance'].median()

print(f"Max distance node1-node2: {max_distance} nm")
print(f"Min distance node1-node2: {min_distance} nm")
print(f"Mean distance node1-node2: {mean_distance} nm")
print(f"Median distance node1-node2: {median_distance} nm")

Max distance node1-node2: 2154.607511822047 nm
Min distance node1-node2: 138.1370696084155 nm
Mean distance node1-node2: 537.9315737284535 nm
Median distance node1-node2: 492.29813121725454 nm


# FRAROBO Volume

In [14]:
pymaid.clear_cache()
frarobo = pymaid.connect_catmaid(server="https://iav-robo.catmaid.virtualflybrain.org/?pid=1", project_id=1, api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {frarobo.catmaid_version}')
neurons_frarobo = pymaid.find_neurons(annotations='Valdes-Aleman et al. 2021')

# Get connectors of neurons of interest
connectors_frarobo = pymaid.get_connectors_between(neurons_frarobo, neurons_frarobo)
connectors_frarobo 

INFO  : Cached cleared. (pymaid)
INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


Server is running CATMAID version 2020.02.15-102-gce1dee4


INFO  : Found 37 neurons matching the search parameters (pymaid)                                                                                                                                                      


connector_id                connector_loc  node1_id  source_neuron  \
0        18924257  [64410.0, 18099.4, 12280.0]  18924256       18924138   
1        18924547  [58793.6, 23225.6, 16840.0]  18924433       18924138   
2        18924605  [59781.6, 22062.8, 16560.0]  18924400       18924138   
3        18924684  [59261.0, 22644.2, 16400.0]  18924450       18924138   
4        18924838  [57581.4, 23962.8, 17640.0]  18924731       18924138   
..            ...                          ...       ...            ...   
823      19317784   [52942.1, 23612.1, 5920.0]  19428371       19395922   
824      19314341   [53050.5, 25552.7, 1720.0]  19428397       19395922   
825      19322619  [32919.9, 22726.8, 11080.0]  19428447       19395922   
826      19322619  [32919.9, 22726.8, 11080.0]  19428447       19395922   
827      19323973  [32928.5, 23424.2, 11600.0]  19428465       19395922   

     confidence1 creator1                    node1_loc  node2_id  \
0              5      NaN  [64356.8, 18183.0, 12280.0]  18924263   
1              5      NaN  [58767.0, 23105.3, 16840.0]  18924567   
2              5      NaN  [60166.7, 21931.1, 16560.0]  19265343   
3              5      NaN  [58998.8, 22566.9, 16400.0]  18924690   
4              5      NaN  [57338.2, 23822.2, 17640.0]  18924865   
..           ...      ...                          ...       ...   
823            5      NaN   [53386.6, 23661.9, 5920.0]  19317785   
824            5      NaN   [53214.3, 25538.2, 1720.0]  19314342   
825            5      NaN  [32773.9, 23007.1, 11080.0]  19322620   
826            5      NaN  [32773.9, 23007.1, 11080.0]  19314727   
827            5      NaN  [32767.7, 23514.1, 11600.0]  19323974   

     target_neuron  confidence2 creator2                    node2_loc  
0         19042468            5      NaN  [64254.2, 17803.0, 12280.0]  
1         19051485            5      NaN  [58786.0, 23324.4, 16720.0]  
2         19051485            5      NaN  [59664.6, 22015.1, 16720.0]  
3         19051485            5      NaN  [59105.2, 22853.2, 16400.0]  
4         19231653            5      NaN  [57619.4, 23909.6, 17520.0]  
..             ...          ...      ...                          ...  
823       19164185            5      NaN   [52713.6, 23613.1, 5920.0]  
824       18934554            5      NaN   [53081.2, 25358.8, 1720.0]  
825       19289102            5      NaN  [34024.4, 22927.4, 11160.0]  
826       18934554            5      NaN  [33040.0, 22595.5, 11160.0]  
827       19289102            5      NaN  [33253.0, 23352.2, 11680.0]  

[828 rows x 12 columns]

In [15]:
# unique confidences for node1 and node2
print(connectors_frarobo["confidence1"].unique())
print(connectors_frarobo["confidence2"].unique())

[5]
[5]


In [16]:
#Find connectors below 100nm

# Compute distances and find those smaller than 100 nm
threshold_nm = 100
distances = connectors_frarobo.apply(lambda row: euclidean_distance(row['connector_loc'], row['node2_loc']), axis=1)
connectors_frarobo.loc[:, 'c_n2_distance'] = distances
small_distances = distances < threshold_nm

# Filter DataFrame for distances smaller than 100 nm
frarobo_filtered_df = connectors_frarobo[small_distances]
print(f"num. of connector-node2 distances smaller than {threshold_nm}nm: {len(frarobo_filtered_df)}")

# Calculate max, min, mean, and median
max_distance = connectors_frarobo['c_n2_distance'].max()
min_distance = connectors_frarobo['c_n2_distance'].min()
mean_distance = connectors_frarobo['c_n2_distance'].mean()
median_distance = connectors_frarobo['c_n2_distance'].median()

print(f"Max distance node1-node2: {max_distance} nm")
print(f"Min distance node1-node2: {min_distance} nm")
print(f"Mean distance node1-node2: {mean_distance} nm")
print(f"Median distance node1-node2: {median_distance} nm")

num. of connector-node2 distances smaller than 100nm: 4
Max distance node1-node2: 1146.965688240064 nm
Min distance node1-node2: 61.88998303441556 nm
Mean distance node1-node2: 327.1402398981908 nm
Median distance node1-node2: 290.6392420523123 nm


In [17]:
# Find distances between node1 and node2

# Compute distances and find those smaller than 100 nm
threshold_nm = 100
distances = connectors_frarobo.apply(lambda row: euclidean_distance(row['node1_loc'], row['node2_loc']), axis=1)
connectors_frarobo.loc[:, 'n1_n2_distance'] = distances
small_distances = distances < threshold_nm

# Filter DataFrame for distances smaller than 100 nm
frarobo_filtered_df = connectors_frarobo[small_distances]
len(frarobo_filtered_df)

# Calculate max, min, mean, and median
max_distance = connectors_frarobo['n1_n2_distance'].max()
min_distance = connectors_frarobo['n1_n2_distance'].min()
mean_distance = connectors_frarobo['n1_n2_distance'].mean()
median_distance = connectors_frarobo['n1_n2_distance'].median()

print(f"Max distance node1-node2: {max_distance} nm")
print(f"Min distance node1-node2: {min_distance} nm")
print(f"Mean distance node1-node2: {mean_distance} nm")
print(f"Median distance node1-node2: {median_distance} nm")

Max distance node1-node2: 1497.948079874597 nm
Min distance node1-node2: 94.47692840053621 nm
Mean distance node1-node2: 486.3699148696125 nm
Median distance node1-node2: 446.9648642015853 nm


In [18]:
# Calculate statistics
def calculate_stats(df, column_name):
    return {
        'mean': df[column_name].mean(),
        'median': df[column_name].median(),
        'max': df[column_name].max(),
        'min': df[column_name].min()
    }

In [45]:
# plot the node1-node2 distances

# # plots in a different tab
# bokeh.io.curdoc().clear()  
# bokeh.io.state.State().reset()
# bokeh.io.reset_output()

# get stats for n1_n2_distance
stats_frarobo = calculate_stats(connectors_frarobo, 'n1_n2_distance')
stats_wt = calculate_stats(connectors_wt, 'n1_n2_distance')
stats_tnt = calculate_stats(connectors_tnt, 'n1_n2_distance')

# Prepare data for Bokeh
categories = ['mean', 'median', 'max', 'min'] 
distances = [stats_wt[cat] for cat in categories]
distances.extend([stats_tnt[cat] for cat in categories])
distances.extend([stats_frarobo[cat] for cat in categories])
print(distances)

data = {
    'categories': categories * 3,
    'Distances': distances,
    'Volume name': ['wt_seymour', 'wt_seymour', 'wt_seymour', 'wt_seymour', 'tnt', 'tnt', 'tnt', 'tnt', 'frarobo', 'frarobo', 'frarobo', 'frarobo'],

    # 'tnt': [stats_tnt[cat] for cat in categories],
    # 'frarobo': [stats_frarobo[cat] for cat in categories],
}

df = pd.DataFrame(data)

display(df)

# Create a factorized DataFrame
factors = [(cat, volume) for cat, volume in zip(df['categories'], df['Volume name'])]

# Prepare the ColumnDataSource
source = ColumnDataSource(data=dict(x=factors, values=df['Distances']))


p = figure(x_range=FactorRange(*factors), height=500, width=900, title="Node1 (pre)- Node2 (post) Distance Statistics by Volume",
           toolbar_location=None, tools="")

# Add bars with legend labels
colors = ['#c9d9d3', '#718dbf', '#e84d60']
p.vbar(x='x', top='values', width=1, source=source, line_color='white',
       fill_color=factor_cmap('x', palette=colors, factors=['wt_seymour', 'tnt', 'frarobo'], start=1, end=2))

# Customize plot
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = 1
p.xaxis.group_label_orientation = "vertical"
p.xaxis.subgroup_label_orientation = "horizontal"
p.outline_line_color = None
p.yaxis.axis_label = 'distance (nm)'

# Add legend manually
# legend = Legend(items=[
#     ("frarobo", [p.renderers[0]]),
#     ("wt_seymour", [p.renderers[0]]),
#     ("tnt", [p.renderers[0]])
# ], location="top_left", title="Volume name")

# p.add_layout(legend, 'right')

# Show the plot
show(p)

# save the results to a file
export_png(p, filename="/Users/sam/Documents/LMB/synful/hemibrain_synapses/node1_node2_distances.png")


[442.0992777118431, 406.60748886364553, 1698.6964325623323, 89.30000000000291, 537.9315737284535, 492.29813121725454, 2154.607511822047, 138.1370696084155, 486.3699148696125, 446.9648642015853, 1497.948079874597, 94.47692840053621]


categories    Distances Volume name
0        mean   442.099278  wt_seymour
1      median   406.607489  wt_seymour
2         max  1698.696433  wt_seymour
3         min    89.300000  wt_seymour
4        mean   537.931574         tnt
5      median   492.298131         tnt
6         max  2154.607512         tnt
7         min   138.137070         tnt
8        mean   486.369915     frarobo
9      median   446.964864     frarobo
10        max  1497.948080     frarobo
11        min    94.476928     frarobo

'/Users/sam/Documents/LMB/synful/hemibrain_synapses/node1_node2_distances.png'

In [54]:
# plot the connector to node 2 distances
stats_frarobo = calculate_stats(connectors_frarobo, 'c_n2_distance')
stats_wt = calculate_stats(connectors_wt, 'c_n2_distance')
stats_tnt = calculate_stats(connectors_tnt, 'c_n2_distance')

# Prepare data for Bokeh
categories = ['mean', 'median', 'max', 'min'] 
distances = [stats_wt[cat] for cat in categories]
distances.extend([stats_tnt[cat] for cat in categories])
distances.extend([stats_frarobo[cat] for cat in categories])
print(distances)

data = {
    'categories': categories * 3,
    'Distances': distances,
    'Volume name': ['wt_seymour', 'wt_seymour', 'wt_seymour', 'wt_seymour', 'tnt', 'tnt', 'tnt', 'tnt', 'frarobo', 'frarobo', 'frarobo', 'frarobo'],

    # 'tnt': [stats_tnt[cat] for cat in categories],
    # 'frarobo': [stats_frarobo[cat] for cat in categories],
}

df = pd.DataFrame(data)

print(df)

# Create a factorized DataFrame
factors = [(cat, volume) for cat, volume in zip(df['categories'], df['Volume name'])]

# Prepare the ColumnDataSource
source = ColumnDataSource(data=dict(x=factors, values=df['Distances']))


# Prepare the Figure
p = figure(x_range=FactorRange(*factors), height=400, width=800, title="Connector - Node2 (post) Distance Statistics by Volume",
           toolbar_location=None, tools="")

# Add bars with legend labels
colors = ['#c9d9d3', '#718dbf', '#e84d60']
p.vbar(x='x', top='values', width=1, source=source, line_color='white',
       fill_color=factor_cmap('x', palette=colors, factors=['wt_seymour', 'tnt', 'frarobo'], start=1, end=2))

# Customize plot
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.xaxis.major_label_orientation = 1
p.xaxis.group_label_orientation = "horizontal"
p.xaxis.subgroup_label_orientation = "horizontal"
p.outline_line_color = None
p.yaxis.axis_label = 'distance (nm)'

# Add legend manually
# legend = Legend(items=[
#     ("frarobo", [p.renderers[0]]),
#     ("wt_seymour", [p.renderers[0]]),
#     ("tnt", [p.renderers[0]])
# ], location="top_left", title="Volume name")

# p.add_layout(legend, 'right')

# Show the plot
show(p)

# save the results to a file
export_png(p, filename="/Users/sam/Documents/LMB/synful/hemibrain_synapses/connectors_node2_distances.png")


[291.72671731101406, 258.91319008501614, 1553.487872498528, 65.74990494289328, 357.2093880866985, 320.65671675484936, 1613.7975151796486, 86.72266139827751, 327.1402398981908, 290.6392420523123, 1146.965688240064, 61.88998303441556]
   categories    Distances Volume name
0        mean   291.726717  wt_seymour
1      median   258.913190  wt_seymour
2         max  1553.487872  wt_seymour
3         min    65.749905  wt_seymour
4        mean   357.209388         tnt
5      median   320.656717         tnt
6         max  1613.797515         tnt
7         min    86.722661         tnt
8        mean   327.140240     frarobo
9      median   290.639242     frarobo
10        max  1146.965688     frarobo
11        min    61.889983     frarobo


'/Users/sam/Documents/LMB/synful/hemibrain_synapses/connectors_node2_distances.png'